**Creating 1-r map**
- Allows for more straightforward visualization of the brain map
- Masked: brain_maps/gsp1000MF_vs_yeo1000_precomputed_avgR-1-r_masked.nii.gz
- Unmasked: brain_maps/gsp1000MF_vs_yeo1000_precomputed_avgR-1-r_dil.nii.gz

In [ ]:
import os
import pandas as pd
from nimlab.jax_functions import NiftiMasker
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

map_path = 'brain_maps/gsp1000MF_vs_yeo1000_precomputed_avgr.nii.gz'
masker = NiftiMasker('MNI152_T1_2mm_brain_mask')
data = masker.transform(map_path)
data = 1-data
new_map = masker.inverse_transform(data)
new_map.to_filename('brain_maps/gsp1000MF_vs_yeo1000_precomputed_avgR-1-r_masked.nii.gz')

masker = NiftiMasker('MNI152_T1_2mm_brain_mask_dil')
data = masker.transform(map_path)
data = 1-data
new_map = masker.inverse_transform(data)
new_map.to_filename('brain_maps/gsp1000MF_vs_yeo1000_precomputed_avgR-1-r_dil.nii.gz')

**Create 1-r map for gsp803/yeo803 differences**

In [ ]:
import os
import pandas as pd
from nimlab.jax_functions import NiftiMasker
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

map_path = 'brain_maps/gsp803_vs_yeo803_precomputed_avgr.nii.gz'
masker = NiftiMasker('MNI152_T1_2mm_brain_mask')
data = masker.transform(map_path)
data = 1-data
new_map = masker.inverse_transform(data)
new_map.to_filename('brain_maps/gsp803_vs_yeo803_precomputed_avgr_avgR-1-r_masked.nii.gz')

masker = NiftiMasker('MNI152_T1_2mm_brain_mask_dil')
data = masker.transform(map_path)
data = 1-data
new_map = masker.inverse_transform(data)
new_map.to_filename('brain_maps/gsp803_vs_yeo803_precomputed_avgr-1-r_dil.nii.gz')

**Creating ROI of voxels with highest 1-r values** (highest disagreement between connectomes) in brainstem.
We are interested in voxels with r less than .90, that are in brainstem/cerebellum.
We will generate three ROIs:
- Values with r < .90 within the brain itself (rois/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi.nii.gz)
- Values with r < .90 within the dilated mask (rois/1-r_01_gsp1000MF_vs_yeo1000_dil_brainstem_roi.nii.gz)
- Values with r < .90 within the dilated mask but outside the brain (csf) (rois/1-r_01_gsp1000MF_vs_yeo1000_dil_brainstem_csf_roi.nii.gz)

In [ ]:
import os
import pandas as pd
from nimlab.jax_functions import NiftiMasker, load_image, load_mask
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

# Masked ROI generation
map_path = 'brain_maps/gsp1000MF_vs_yeo1000_precomputed_avgR-1-r_masked.nii.gz'
map_data = load_image(map_path).get_fdata()

# Exclude anything in map_image superior to MNI z=-32 (in array space, anything above 20)
map_data[:, :, 20:] = 0
# Exclude anything in map_image anterior to y=-34 (in array space, anything above 46)
map_data[:, 46:, :] = 0

# Threshold and binarize at 0.1
map_data[map_data < 0.1] = 0
map_data[map_data >= 0.1] = 1

masker = NiftiMasker('MNI152_T1_2mm_brain_mask')
new_image = masker.inverse_transform(masker.transform(load_image(map_data)))

new_image.to_filename('rois/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi.nii.gz')


# Dilated ROI generation
map_path = 'brain_maps/gsp1000MF_vs_yeo1000_precomputed_avgR-1-r_dil.nii.gz'
map_data = load_image(map_path).get_fdata()

# Exclude anything in map_image superior to MNI z=-32 (in array space, anything above 20)
map_data[:, :, 20:] = 0
# Exclude anything in map_image anterior to y=-34 (in array space, anything above 46)
map_data[:, 46:, :] = 0

# Threshold and binarize at 0.1
map_data[map_data < 0.1] = 0
map_data[map_data >= 0.1] = 1

masker = NiftiMasker('MNI152_T1_2mm_brain_mask_dil')
new_image = masker.inverse_transform(masker.transform(load_image(map_data)))

new_image.to_filename('rois/1-r_01_gsp1000MF_vs_yeo1000_dil_brainstem_roi.nii.gz')

# Find the voxels in the dilated ROI that are not in the masked ROI
dilated_roi = masker.transform('rois/1-r_01_gsp1000MF_vs_yeo1000_dil_brainstem_roi.nii.gz')
masked_roi = masker.transform('rois/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi.nii.gz')
diff_roi = dilated_roi - masked_roi
diff_roi_image = masker.inverse_transform(diff_roi)
diff_roi_image.to_filename('rois/1-r_01_gsp1000MF_vs_yeo1000_csf_brainstem_roi.nii.gz')

**Sampling spheres from the ROIs**
- We are going to make 15 spheres of 2mm radius across the discrepant voxels.
- 10 from the brain, 5 from the CSF
- We will save the coordinates and paths to the spheres in csvs/sphere_coords.csv
- The spheres will be saved in /rois

In [ ]:
from nilearn import image, plotting
from nimlab.jax_functions import load_mask, load_image
import os
import numpy as np
from numpy.linalg import inv
import pandas as pd

if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

def make_sphere(coord, brain_mask, radius, voxel_coord = False):
    #Transform from MNI coordinates to voxelwise(matrix) coords
    if voxel_coord == False:
        inv_affine = inv(brain_mask.affine)
        
        trans_raw_coord = image.coord_transform(coord[0], coord[1], coord[2], inv_affine)
        trans_coord = round(trans_raw_coord[0]), round(trans_raw_coord[1]), round(trans_raw_coord[2])

    else:
        trans_coord = coord

    bin_sphere = create_bin_sphere(brain_mask.shape, trans_coord, radius)
    sphere_img = image.new_img_like(brain_mask, bin_sphere)
    return sphere_img

def create_bin_sphere(arr_size, center, r):
    # https://stackoverflow.com/questions/53326570/how-to-create-sphere-inside-a-ndarray-python?noredirect=1&lq=1
    coords = np.ogrid[:arr_size[0], :arr_size[1], :arr_size[2]]
    distance = np.sqrt((coords[0] - center[0])**2 + (coords[1]-center[1])**2 + (coords[2]-center[2])**2) 
    return 1*(distance <= r)

def save_sphere(coord):
    mask = load_mask('MNI152_T1_2mm_brain_mask_dil')
    sphere_img = make_sphere(coord, mask, 2)
    filename = f'rois/sphere_{coord[0]}_{coord[1]}_{coord[2]}.nii.gz'
    sphere_img.to_filename(filename)
    return filename

# Load the masked ROI
masked_roi = load_image('rois/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi.nii.gz')
masked_roi_data = masked_roi.get_fdata()

# Load the CSF ROI
csf_roi = load_image('rois/1-r_01_gsp1000MF_vs_yeo1000_csf_brainstem_roi.nii.gz')
csf_roi_data = csf_roi.get_fdata()

# Now, we want to randomly select the indices of 10 voxels in the masked ROI where the value is 1
masked_roi_indices = np.argwhere(masked_roi_data == 1)
np.random.shuffle(masked_roi_indices)
masked_roi_indices = masked_roi_indices[:20]

# We want to do the same thing with the CSF ROI, but only 5 voxels
csf_roi_indices = np.argwhere(csf_roi_data == 1)
np.random.shuffle(csf_roi_indices)
csf_roi_indices = csf_roi_indices[:10]

# Create lists to hold the data
brain_data = []
csf_data = []

# Add brain data to the list
for i in range(20):
    mni_coord = image.coord_transform(masked_roi_indices[i][0], masked_roi_indices[i][1], masked_roi_indices[i][2], masked_roi.affine)
    mni_coord = round(mni_coord[0]), round(mni_coord[1]), round(mni_coord[2])
    brain_data.append({'region_type': 'brain', 'voxel_coord': tuple(masked_roi_indices[i]), 'mni_coord': mni_coord})

# Add CSF data to the list
for i in range(10):
    mni_coord = image.coord_transform(csf_roi_indices[i][0], csf_roi_indices[i][1], csf_roi_indices[i][2], csf_roi.affine)
    mni_coord = round(mni_coord[0]), round(mni_coord[1]), round(mni_coord[2])
    csf_data.append({'region_type': 'csf', 'voxel_coord': tuple(csf_roi_indices[i]), 'mni_coord': mni_coord})

# Convert lists to DataFrames
brain_df = pd.DataFrame(brain_data)
csf_df = pd.DataFrame(csf_data)
df = pd.concat([brain_df, csf_df], ignore_index=True)

df['sphere_path'] = df['mni_coord'].apply(save_sphere)
df.to_csv('csvs/sphere_coords.csv', index=False)

Let's just plot a sphere using nilearn to see if it worked

In [ ]:
from nilearn import image, plotting
from nimlab.jax_functions import load_mask, load_image
import os
import numpy as np
from numpy.linalg import inv
import pandas as pd

if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

df = pd.read_csv('csvs/sphere_coords.csv')
sphere_path = df.loc[0, 'sphere_path']
plotting.plot_roi(sphere_path, title = '5mm sphere around MNI (0,-44,-58)', display_mode='ortho', cut_coords=[0, -44, -58], cmap='Reds')

Create the GSP1000_MF Networks for each sphere using precomputed connectome

In [ ]:
# from nilearn import image, plotting
# from nimlab.jax_functions import load_mask, load_image
import os
import numpy as np
from numpy.linalg import inv
import pandas as pd

if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

# Load the DataFrame
df = pd.read_csv('csvs/sphere_coords.csv')

if 't' not in df.columns:
    df['t'] = 'None'
    
# Function to ensure the directory exists and to construct the output path correctly
def create_output_path(connectome_name):
    connectivity_dir = os.path.abspath(f"{connectome_name}_networks")
    if not os.path.exists(connectivity_dir):
        os.makedirs(connectivity_dir)
    return connectivity_dir

# Iterate through each row in the DataFrame
connectome_name = "GSP1000_MF"
for i, row in df.iterrows():
    roi = row['sphere_path']    
    # Create the output path
    output_path = create_output_path(connectome_name)
    hypothetical_path = os.path.abspath(os.path.join(output_path, os.path.basename(f'{roi.replace(".nii.gz","")}_Precom_T.nii.gz')))
    df.at[i, 't'] = hypothetical_path
    df.at[i, 'sphere_path'] = os.path.abspath(roi)
    if os.path.exists(hypothetical_path):
        print(f"Skipping {hypothetical_path} because it already exists.")
        continue
    
    # Construct the system command
    command = f'connectome_precomputed -r {os.path.dirname(roi)} -c eris-{connectome_name} -o {output_path}'
    
    # Print the command for debugging purposes
    print(f'Executing command: {command}')
    
    # Execute the command
    os.system(command)
    
    # Check if the output file is being created correctly
    if not os.path.exists(output_path):
        print(f"Error: The expected output file {output_path} was not created.")

df.to_csv('csvs/sphere_coords.csv', index=False)

Create the Yeo1000_dil networks for each sphere using precomputed connectome

In [ ]:
# from nilearn import image, plotting
# from nimlab.jax_functions import load_mask, load_image
import os
import numpy as np
from numpy.linalg import inv
import pandas as pd

if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

# Load the DataFrame
df = pd.read_csv('csvs/sphere_coords.csv')

if 't' not in df.columns:
    df['t'] = 'None'
    
# Function to ensure the directory exists and to construct the output path correctly
def create_output_path(connectome_name):
    connectivity_dir = os.path.abspath(f"{connectome_name}_networks")
    if not os.path.exists(connectivity_dir):
        os.makedirs(connectivity_dir)
    return connectivity_dir

# Iterate through each row in the DataFrame
connectome_name = "yeo1000_dil"
for i, row in df.iterrows():
    roi = row['sphere_path']    
    # Create the output path
    output_path = create_output_path(connectome_name)
    hypothetical_path = os.path.abspath(os.path.join(output_path, os.path.basename(f'{roi.replace(".nii.gz","")}_Precom_T.nii.gz')))
    df.at[i, 't'] = hypothetical_path
    df.at[i, 'sphere_path'] = os.path.abspath(roi)
    if os.path.exists(hypothetical_path):
        print(f"Skipping {hypothetical_path} because it already exists.")
        continue
    
    # Construct the system command
    command = f'connectome_precomputed -r {os.path.dirname(roi)} -c eris-{connectome_name} -o {output_path}'
    
    # Print the command for debugging purposes
    print(f'Executing command: {command}')
    
    # Execute the command
    os.system(command)
    
    # Check if the output file is being created correctly
    if not os.path.exists(output_path):
        print(f"Error: The expected output file {output_path} was not created.")

df.to_csv('csvs/sphere_coords.csv', index=False)

Let's generate the network maps for the larger ROIs using regular connectome, not the precomputed connectome (GSP1000_MF)

In [ ]:
from __future__ import print_function
import os
import numpy as np
from glob import glob
from natsort import natsorted
from tqdm import tqdm
from nimlab import connectomics as cs
from nimlab.connectomics import ConnectomeSubject, calculate_maps
from nilearn.input_data import NiftiMasker
# from nimlab.jax_functions import NiftiMasker
from multiprocessing import Pool, Queue, Process
from collections import OrderedDict
from scipy.stats import ttest_1samp
from nimlab import datasets as ds
from scipy import stats
from numba import jit

if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')


def generate_t_maps(
    cs_roi,
    cs_brain,
    roi_list,
    roi_connectome_type='volume',
    brain_connectome_type='volume',
    same_connectome=False,
    warning_flag=True,
    num_workers=4,
    output_folder='/path/to/output/folder',
    ind_file_output="",
    single_connectome_subject=False
):
    # Create ConnectomeSubject objects
    subjects = create_connectome_subjects(
        cs_roi, cs_brain, roi_list, [], roi_connectome_type, brain_connectome_type, same_connectome, warning_flag
    )

    # Calculate AvgR_Fz, AvgR, and T maps
    avgR_fz_maps, avgR_maps, T_maps = calculate_maps(
        subjects, num_workers, output_folder, ind_file_output, single_connectome_subject
    )

    return avgR_fz_maps, avgR_maps, T_maps


def create_connectome_subjects(cs_roi, cs_brain, roi_list, masker_list, roi_connectome_type, brain_connectome_type, same_connectome, warning_flag):
    # Load paths to all ROI connectome files
    roi_connectome_files_norms = natsorted(glob(os.path.join(cs_roi, "*_norms.npy")))
    roi_connectome_files = [
        (glob(f.split("_norms")[0] + ".npy")[0], f) for f in roi_connectome_files_norms
    ]

    # Load paths to all Brain connectome files
    brain_connectome_files_norms = natsorted(glob(os.path.join(cs_brain, "*_norms.npy")))
    brain_connectome_files = [
        (glob(f.split("_norms")[0] + ".npy")[0], f)
        for f in brain_connectome_files_norms
    ]

    # Create ConnectomeSubject objects
    subjects = []
    for roi_connectome_file, brain_connectome_file in zip(roi_connectome_files, brain_connectome_files):
        subject = ConnectomeSubject(
            roi_connectome_file,
            brain_connectome_file,
            roi_list,
            roi_connectome_type,
            brain_connectome_type,
            same_connectome,
            warning_flag
        )
        subjects.append(subject)
    
    return subjects

cs_roi = '/data/nimlab/connectome_npy/GSP1000_MF'
cs_brain = '/data/nimlab/connectome_npy/GSP1000_MF'
roi_connectome_type = 'volume'
brain_connectome_type = 'volume'
same_connectome = False
warning_flag = True
num_workers = 4
output_folder = '/PHShome/jt041/projects/connectome_differences/GSP1000_MF_networks'
ind_file_output = ""
single_connectome_subject = False
mask = ds.get_img("MNI152_T1_2mm_brain_mask_dil")

roi_filename_dict = {
    0: 'rois/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi.nii.gz',
    1: 'rois/1-r_01_gsp1000MF_vs_yeo1000_csf_brainstem_roi.nii.gz',
    2: 'rois/1-r_01_gsp1000MF_vs_yeo1000_dil_brainstem_roi.nii.gz',
    3: 'rois/MRT_bilateral.nii.gz',
    4: 'rois/VSM_bilateral.nii.gz',
}

roi_list = {
    0: np.atleast_2d(NiftiMasker(mask).fit_transform('/PHShome/jt041/projects/connectome_differences/rois_2/1-r_01_gsp1000MF_vs_yeo1000_csf_brainstem_roi.nii.gz')),
    1: np.atleast_2d(NiftiMasker(mask).fit_transform('/PHShome/jt041/projects/connectome_differences/rois_2/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi.nii.gz')),
    2: np.atleast_2d(NiftiMasker(mask).fit_transform('/PHShome/jt041/projects/connectome_differences/rois_2/1-r_01_gsp1000MF_vs_yeo1000_dil_brainstem_roi.nii.gz')),
    3: np.atleast_2d(NiftiMasker(mask).fit_transform('/PHShome/jt041/projects/connectome_differences/rois_2/MRT_bilateral.nii.gz')),
    4: np.atleast_2d(NiftiMasker(mask).fit_transform('/PHShome/jt041/projects/connectome_differences/rois_2/VSM_bilateral.nii.gz')),
}

avgR_fz_maps, avgR_maps, T_maps = generate_t_maps(
    cs_roi, cs_brain, roi_list, roi_connectome_type, brain_connectome_type, same_connectome, warning_flag, num_workers, output_folder, ind_file_output, single_connectome_subject
)

roi_filename_dict = {
    0: 'GSP1000_MF_networks/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi.nii.gz',
    1: 'GSP1000_MF_networks/1-r_01_gsp1000MF_vs_yeo1000_csf_brainstem_roi.nii.gz',
    2: 'GSP1000_MF_networks/1-r_01_gsp1000MF_vs_yeo1000_dil_brainstem_roi.nii.gz',
    3: 'GSP1000_MF_networks/MRT_bilateral.nii.gz',
    4: 'GSP1000_MF_networks/VSM_bilateral.nii.gz',
}

for i in range(len(T_maps)):
    masker = NiftiMasker(mask).fit()
    t_map = masker.inverse_transform(T_maps[i])
    filename = roi_filename_dict[i].replace('.nii.gz', '_T.nii.gz')
    t_map.to_filename(os.path.abspath(filename))

for i in range(len(avgR_maps)):
    masker = NiftiMasker(mask).fit()
    avgR_map = masker.inverse_transform(avgR_maps[i])
    filename = roi_filename_dict[i].replace('.nii.gz', '_AvgR.nii.gz')
    avgR_map.to_filename(os.path.abspath(filename))

for i in range(len(avgR_fz_maps)):
    masker = NiftiMasker(mask).fit()
    avgR_fz_map = masker.inverse_transform(avgR_fz_maps[i])
    filename = roi_filename_dict[i].replace('.nii.gz', '_AvgR_Fz.nii.gz')
    avgR_fz_map.to_filename(os.path.abspath(filename))

Now, let's generate the network maps for the larger ROIs using regular connectome, not the precomputed connectome (Yeo1000_dil)

In [ ]:
from __future__ import print_function
import os
import numpy as np
from glob import glob
from natsort import natsorted
from tqdm import tqdm
from nimlab import connectomics as cs
from nimlab.connectomics import ConnectomeSubject, calculate_maps
from nilearn.input_data import NiftiMasker
# from nimlab.jax_functions import NiftiMasker
from multiprocessing import Pool, Queue, Process
from collections import OrderedDict
from scipy.stats import ttest_1samp
from nimlab import datasets as ds
from scipy import stats
from numba import jit

if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')


def generate_t_maps(
    cs_roi,
    cs_brain,
    roi_list,
    roi_connectome_type='volume',
    brain_connectome_type='volume',
    same_connectome=False,
    warning_flag=True,
    num_workers=4,
    output_folder='/path/to/output/folder',
    ind_file_output="",
    single_connectome_subject=False
):
    # Create ConnectomeSubject objects
    subjects = create_connectome_subjects(
        cs_roi, cs_brain, roi_list, [], roi_connectome_type, brain_connectome_type, same_connectome, warning_flag
    )

    # Calculate AvgR_Fz, AvgR, and T maps
    avgR_fz_maps, avgR_maps, T_maps = calculate_maps(
        subjects, num_workers, output_folder, ind_file_output, single_connectome_subject
    )

    return avgR_fz_maps, avgR_maps, T_maps


def create_connectome_subjects(cs_roi, cs_brain, roi_list, masker_list, roi_connectome_type, brain_connectome_type, same_connectome, warning_flag):
    # Load paths to all ROI connectome files
    roi_connectome_files_norms = natsorted(glob(os.path.join(cs_roi, "*_norms.npy")))
    roi_connectome_files = [
        (glob(f.split("_norms")[0] + ".npy")[0], f) for f in roi_connectome_files_norms
    ]

    # Load paths to all Brain connectome files
    brain_connectome_files_norms = natsorted(glob(os.path.join(cs_brain, "*_norms.npy")))
    brain_connectome_files = [
        (glob(f.split("_norms")[0] + ".npy")[0], f)
        for f in brain_connectome_files_norms
    ]

    # Create ConnectomeSubject objects
    subjects = []
    for roi_connectome_file, brain_connectome_file in zip(roi_connectome_files, brain_connectome_files):
        subject = ConnectomeSubject(
            roi_connectome_file,
            brain_connectome_file,
            roi_list,
            roi_connectome_type,
            brain_connectome_type,
            same_connectome,
            warning_flag
        )
        subjects.append(subject)
    
    return subjects

cs_roi = '/data/nimlab/connectome_npy/yeo1000_dil'
cs_brain = '/data/nimlab/connectome_npy/yeo1000_dil'
roi_connectome_type = 'volume'
brain_connectome_type = 'volume'
same_connectome = False
warning_flag = True
num_workers = 4
output_folder = '/PHShome/jt041/projects/connectome_differences/yeo1000_dil_networks'
ind_file_output = ""
single_connectome_subject = False
mask = ds.get_img("MNI152_T1_2mm_brain_mask_dil")

roi_filename_dict = {
    0: 'rois/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi.nii.gz',
    1: 'rois/1-r_01_gsp1000MF_vs_yeo1000_csf_brainstem_roi.nii.gz',
    2: 'rois/1-r_01_gsp1000MF_vs_yeo1000_dil_brainstem_roi.nii.gz',
    3: 'rois/MRT_bilateral.nii.gz',
    4: 'rois/VSM_bilateral.nii.gz',
}

roi_list = {
    0: np.atleast_2d(NiftiMasker(mask).fit_transform('/PHShome/jt041/projects/connectome_differences/rois_2/1-r_01_gsp1000MF_vs_yeo1000_csf_brainstem_roi.nii.gz')),
    1: np.atleast_2d(NiftiMasker(mask).fit_transform('/PHShome/jt041/projects/connectome_differences/rois_2/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi.nii.gz')),
    2: np.atleast_2d(NiftiMasker(mask).fit_transform('/PHShome/jt041/projects/connectome_differences/rois_2/1-r_01_gsp1000MF_vs_yeo1000_dil_brainstem_roi.nii.gz')),
    3: np.atleast_2d(NiftiMasker(mask).fit_transform('/PHShome/jt041/projects/connectome_differences/rois_2/MRT_bilateral.nii.gz')),
    4: np.atleast_2d(NiftiMasker(mask).fit_transform('/PHShome/jt041/projects/connectome_differences/rois_2/VSM_bilateral.nii.gz')),
}

avgR_fz_maps, avgR_maps, T_maps = generate_t_maps(
    cs_roi, cs_brain, roi_list, roi_connectome_type, brain_connectome_type, same_connectome, warning_flag, num_workers, output_folder, ind_file_output, single_connectome_subject
)

roi_filename_dict = {
    0: 'yeo1000_dil_networks/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi.nii.gz',
    1: 'yeo1000_dil_networks/1-r_01_gsp1000MF_vs_yeo1000_csf_brainstem_roi.nii.gz',
    2: 'yeo1000_dil_networks/1-r_01_gsp1000MF_vs_yeo1000_dil_brainstem_roi.nii.gz',
    3: 'yeo1000_dil_networks/MRT_bilateral.nii.gz',
    4: 'yeo1000_dil_networks/VSM_bilateral.nii.gz',
}

for i in range(len(T_maps)):
    masker = NiftiMasker(mask).fit()
    t_map = masker.inverse_transform(T_maps[i])
    filename = roi_filename_dict[i].replace('.nii.gz', '_T.nii.gz')
    t_map.to_filename(os.path.abspath(filename))

for i in range(len(avgR_maps)):
    masker = NiftiMasker(mask).fit()
    avgR_map = masker.inverse_transform(avgR_maps[i])
    filename = roi_filename_dict[i].replace('.nii.gz', '_AvgR.nii.gz')
    avgR_map.to_filename(os.path.abspath(filename))

for i in range(len(avgR_fz_maps)):
    masker = NiftiMasker(mask).fit()
    avgR_fz_map = masker.inverse_transform(avgR_fz_maps[i])
    filename = roi_filename_dict[i].replace('.nii.gz', '_AvgR_Fz.nii.gz')
    avgR_fz_map.to_filename(os.path.abspath(filename))

**Create a master csv that contains all the information for the spheres**

In [ ]:
import os
import pandas as pd

# Adjust the working directory if running from 'notebooks'
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

# Read the initial sphere coordinates CSV
df = pd.read_csv('csvs/sphere_coords.csv')

# Initialize the master dataframe
master_df = pd.DataFrame(columns=['region_type', 'sphere', 'voxel_coord', 'mni_coord', 'roi_path', 't_gsp1000MF', 't_yeo1000_dil'])

# Iterate through the initial dataframe to populate the master dataframe
rows_list = []
for index, row in df.iterrows():
    region_type = row.get('region_type', 'brain')  # Default to 'brain' if not provided
    sphere = index < 30  # First 30 rows have sphere as True, rest as False
    voxel_coord = row.get('voxel_coord') if index < 30 else None
    mni_coord = row.get('mni_coord') if index < 30 else None
    roi_path = row['sphere_path']

    # Construct the file paths for the t_gsp1000MF and t_yeo1000_dil columns
    roi_basename = os.path.basename(roi_path).replace('.nii.gz', '')
    if roi_basename in [
        'MRT_bilateral', 'VSM_bilateral',
        '1-r_01_gsp1000MF_vs_yeo1000_dil_brainstem_roi',
        '1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi',
        '1-r_01_gsp1000MF_vs_yeo1000_csf_brainstem_roi'
    ]:
        t_gsp1000MF_path = f'/PHShome/jt041/projects/connectome_differences/GSP1000_MF_networks/{roi_basename}_T.nii.gz'
        t_yeo1000_dil_path = f'/PHShome/jt041/projects/connectome_differences/yeo1000_dil_networks/{roi_basename}_T.nii.gz'
    else:
        t_gsp1000MF_path = f'/PHShome/jt041/projects/connectome_differences/GSP1000_MF_networks/{roi_basename}_Precom_T.nii.gz'
        t_yeo1000_dil_path = f'/PHShome/jt041/projects/connectome_differences/yeo1000_dil_networks/{roi_basename}_Precom_T.nii.gz'

    # Create a dictionary for the row
    row_dict = {
        'region_type': region_type,
        'sphere': sphere,
        'voxel_coord': voxel_coord,
        'mni_coord': mni_coord,
        'roi_path': roi_path,
        't_gsp1000MF': t_gsp1000MF_path,
        't_yeo1000_dil': t_yeo1000_dil_path
    }
    
    # Append the dictionary to the list of rows
    rows_list.append(row_dict)

# Concatenate the rows list into the master dataframe
master_df = pd.concat([master_df, pd.DataFrame(rows_list)], ignore_index=True)

# Optionally save the master dataframe to a new CSV file
master_df.to_csv('csvs/full_file_list.csv', index=False)

master_df

Load up the CSF mask and create some variants

In [5]:
import os
import pandas as pd
from nimlab.jax_functions import NiftiMasker, load_mask, load_image
import nibabel as nib
from nilearn.image import resample_to_img
import numpy as np
import numpy as np
from scipy.ndimage import binary_fill_holes


# Adjust the working directory if running from 'notebooks'
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

# csf_mask_path = 'masks/csf_mask.nii.gz'
brain_mask ='MNI152_T1_2mm_brain_mask'
# brain_mask_img = load_mask(brain_mask)

# csf_img = nib.load(csf_mask_path)

# # Now, resample csf_img to the brain_mask_img and save as csf_mask_2mm.nii.gz (using nearest neighbor interpolation)
# resampled_csf_img = resample_to_img(csf_img, brain_mask_img, interpolation='nearest')
# resampled_csf_img.to_filename('masks/csf_mask_2mm.nii.gz')

brain_mask_data = load_mask(brain_mask).get_fdata()
brain_mask_data = binary_fill_holes(brain_mask_data).astype(float)
brain_mask_img = load_image(brain_mask_data)

masker = NiftiMasker(brain_mask_img)


csf_mask_path = 'masks/csf_mask_2mm.nii.gz'

mask_data = masker.transform(csf_mask_path)
internal_csf_img = masker.inverse_transform(mask_data)
internal_csf_img.to_filename('masks/internal_csf_mask_2mm.nii.gz')

intern_csf_img_array = internal_csf_img.get_fdata()

# We're gonna chop this array into a few pieces. 
# piece A: x (anything), y (min:52), z (min:24) # We'll call this brainstem csf
# piece B: x (anything), y (52: max), z (24:34) # We'll call this inferomedial csf
# piece C: x (anything), y (min:52), z (24:38) # We'll call this posterior csf
# Piece D: x (anything), y (anything), z (34:52) # We'll call this lateral ventricular csf

brainstem_csf = np.zeros_like(intern_csf_img_array)
inferomedial_csf = np.zeros_like(intern_csf_img_array)
posterior_csf = np.zeros_like(intern_csf_img_array)
lateral_ventricular_csf = np.zeros_like(intern_csf_img_array)

brainstem_csf[:, :, 24:52] = intern_csf_img_array[:, :, 24:52]
inferomedial_csf[:, 52:, 24:34] = intern_csf_img_array[:, 52:, 24:34]
posterior_csf[:, :, 24:38] = intern_csf_img_array[:, :, 24:38]
lateral_ventricular_csf[:, :, 34:52] = intern_csf_img_array[:, :, 34:52]

# Save everything to a new file
load_image(brainstem_csf).to_filename('masks/brainstem_csf_mask_2mm.nii.gz')
load_image(inferomedial_csf).to_filename('masks/inferomedial_csf_mask_2mm.nii.gz')
load_image(posterior_csf).to_filename('masks/posterior_csf_mask_2mm.nii.gz')
load_image(lateral_ventricular_csf).to_filename('masks/lateral_ventricular_csf_mask_2mm.nii.gz')

dilated_masker = NiftiMasker('MNI152_T1_2mm_brain_mask_dil')
external_csf = dilated_masker.transform('masks/csf_mask_2mm.nii.gz')
internal_csf = dilated_masker.transform('masks/internal_csf_mask_2mm.nii.gz')
external_csf = external_csf - internal_csf
external_csf = dilated_masker.inverse_transform(external_csf)

load_image(external_csf).to_filename('masks/external_csf_mask_2mm.nii.gz')


**Now, let's randomly sample some voxels within each CSF mask, and save the spheres and their coordinates to a dataframe and directory**

In [6]:
import os
import numpy as np
import pandas as pd
from nilearn import image
from nimlab.jax_functions import load_mask, load_image
from numpy.linalg import inv

# Ensure we're in the correct directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

save_dir = "csf_spheres_rois"
csf_masks = [
    'masks/brainstem_csf_mask_2mm.nii.gz',
    'masks/inferomedial_csf_mask_2mm.nii.gz',
    'masks/posterior_csf_mask_2mm.nii.gz',
    'masks/lateral_ventricular_csf_mask_2mm.nii.gz',
    'masks/external_csf_mask_2mm.nii.gz'
]

# Create the save directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

def make_sphere(coord, brain_mask, radius, voxel_coord=False):
    if not voxel_coord:
        inv_affine = inv(brain_mask.affine)
        trans_raw_coord = image.coord_transform(coord[0], coord[1], coord[2], inv_affine)
        trans_coord = round(trans_raw_coord[0]), round(trans_raw_coord[1]), round(trans_raw_coord[2])
    else:
        trans_coord = coord

    bin_sphere = create_bin_sphere(brain_mask.shape, trans_coord, radius)
    sphere_img = image.new_img_like(brain_mask, bin_sphere)
    return sphere_img

def create_bin_sphere(arr_size, center, r):
    coords = np.ogrid[:arr_size[0], :arr_size[1], :arr_size[2]]
    distance = np.sqrt((coords[0] - center[0])**2 + (coords[1] - center[1])**2 + (coords[2] - center[2])**2)
    return 1 * (distance <= r)

def save_sphere(coord):
    mask = load_mask('MNI152_T1_2mm_brain_mask_dil')
    sphere_img = make_sphere(coord, mask, 1)
    filename = os.path.join(save_dir, f'sphere_{coord[0]}_{coord[1]}_{coord[2]}.nii.gz')
    sphere_img.to_filename(filename)
    return filename

# Initialize lists to hold data
csf_data = []

# Sample five spheres from each CSF mask
for csf_mask_path in csf_masks:
    csf_mask = load_image(csf_mask_path)
    csf_mask_data = csf_mask.get_fdata()

    # Randomly select the indices of 5 voxels in the CSF mask where the value is 1
    csf_roi_indices = np.argwhere(csf_mask_data == 1)
    np.random.shuffle(csf_roi_indices)
    csf_roi_indices = csf_roi_indices[:5]

    for i in range(5):
        mni_coord = image.coord_transform(csf_roi_indices[i][0], csf_roi_indices[i][1], csf_roi_indices[i][2], csf_mask.affine)
        mni_coord = round(mni_coord[0]), round(mni_coord[1]), round(mni_coord[2])
        csf_data.append({'region_type': 'csf', 'voxel_coord': tuple(csf_roi_indices[i]), 'mni_coord': mni_coord})

# Convert list to DataFrame
csf_df = pd.DataFrame(csf_data)
csf_df['sphere_path'] = csf_df['mni_coord'].apply(save_sphere)
csf_df.to_csv('csvs/csf_sphere_coords.csv', index=False)

print("Sphere generation and saving complete.")

/PHShome/jt041/scratch/ipykernel_132568/2444916713.py:33: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  sphere_img = image.new_img_like(brain_mask, bin_sphere)


Sphere generation and saving complete.


**Make an actually good lateral ventricle mask**

In [4]:
import os
import numpy as np
import pandas as pd
from nilearn import image
from nimlab.jax_functions import load_mask, load_image, NiftiMasker
from numpy.linalg import inv

# Ensure we're in the correct directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

masker = NiftiMasker()
path = 'masks/HarvardOxford-sub-maxprob-thr0-2mm.nii.gz'
data = masker.transform(path)
# Set all values to be 0 except where the values is 3 or the value is 14

# Keep same shape and all values where the value is 3 should be 1, all others 0
left_lateral_ventricle = np.zeros_like(data)
left_lateral_ventricle[data == 3] = 1

# Keep same shape and all values where the value is 14 should be 1, all others 0
right_lateral_ventricle = np.zeros_like(data)
right_lateral_ventricle[data == 14] = 1

left_lateral_ventricle = masker.inverse_transform(left_lateral_ventricle)
right_lateral_ventricle = masker.inverse_transform(right_lateral_ventricle)

left_lateral_ventricle.to_filename('masks/left_lateral_ventricle_mask_2mm.nii.gz')
right_lateral_ventricle.to_filename('masks/right_lateral_ventricle_mask_2mm.nii.gz')

# Now, for both of these ventricle masks, let's cut them in half along the y axis; everything ahead of y=56 will be kept, with suffix appended ('anterior').
# Everything behind y=56 will be kept, with suffix appended ('posterior').

left_lateral_ventricle_data = left_lateral_ventricle.get_fdata()
right_lateral_ventricle_data = right_lateral_ventricle.get_fdata()

left_anterior_lateral_ventricle = np.zeros_like(left_lateral_ventricle_data)
left_posterior_lateral_ventricle = np.zeros_like(left_lateral_ventricle_data)

right_anterior_lateral_ventricle = np.zeros_like(right_lateral_ventricle_data)
right_posterior_lateral_ventricle = np.zeros_like(right_lateral_ventricle_data)

left_anterior_lateral_ventricle[:, :56, :] = left_lateral_ventricle_data[:, :56, :]
left_posterior_lateral_ventricle[:, 56:, :] = left_lateral_ventricle_data[:, 56:, :]

right_anterior_lateral_ventricle[:, :56, :] = right_lateral_ventricle_data[:, :56, :]
right_posterior_lateral_ventricle[:, 56:, :] = right_lateral_ventricle_data[:, 56:, :]

left_anterior_lateral_ventricle = load_image(left_anterior_lateral_ventricle)
left_posterior_lateral_ventricle = load_image(left_posterior_lateral_ventricle)

right_anterior_lateral_ventricle = load_image(right_anterior_lateral_ventricle)
right_posterior_lateral_ventricle = load_image(right_posterior_lateral_ventricle)

left_anterior_lateral_ventricle.to_filename('masks/left_anterior_lateral_ventricle_mask_2mm.nii.gz')
left_posterior_lateral_ventricle.to_filename('masks/left_posterior_lateral_ventricle_mask_2mm.nii.gz')
right_anterior_lateral_ventricle.to_filename('masks/right_anterior_lateral_ventricle_mask_2mm.nii.gz')
right_posterior_lateral_ventricle.to_filename('masks/right_posterior_lateral_ventricle_mask_2mm.nii.gz')

**Create a pregenual ACC mask**

In [6]:
import os
import numpy as np
import pandas as pd
from nilearn import image
from nimlab.jax_functions import load_mask, load_image, NiftiMasker
from numpy.linalg import inv
from nilearn.image import resample_to_img, load_img

# Ensure we're in the correct directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

path_1mm = 'masks/AAL3_1mm.nii.gz'
img = load_img(path_1mm)
# We only want to keep values in img where value is 149 or 150. We need to keep the same shape.
# Where its 149 or 150, set to 1, else 0
img_data = img.get_fdata()
new_data = np.zeros_like(img_data)
new_data[(img_data == 149) | (img_data == 150)] = 1
new_img = image.new_img_like(img, new_data)

mask_img = load_mask('MNI152_T1_2mm_brain_mask_dil')

resampled_img = resample_to_img(new_img, mask_img, interpolation='nearest')
resampled_img.to_filename('masks/pregenual.nii.gz')


**Create a subgenual ACC mask**

In [9]:
import os
import numpy as np
import pandas as pd
from nilearn import image
from nimlab.jax_functions import load_mask, load_image, NiftiMasker
from numpy.linalg import inv
from nilearn.image import resample_to_img, load_img

# Ensure we're in the correct directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

path_1mm = 'masks/AAL3_1mm.nii.gz'
img = load_img(path_1mm)
# We only want to keep values in img where value is 149 or 150. We need to keep the same shape.
# Where its 149 or 150, set to 1, else 0
img_data = img.get_fdata()
new_data = np.zeros_like(img_data)
new_data[(img_data == 147) | (img_data == 148)] = 1
new_img = image.new_img_like(img, new_data)

mask_img = load_mask('MNI152_T1_2mm_brain_mask_dil')

resampled_img = resample_to_img(new_img, mask_img, interpolation='nearest')
resampled_img.to_filename('masks/subgenual.nii.gz')


**Merge pregenual and subgenual ACC masks**

In [10]:
import os
import os
import numpy as np
import pandas as pd
from nilearn import image
from nimlab.jax_functions import load_mask, load_image, NiftiMasker
from numpy.linalg import inv
from nilearn.image import resample_to_img, load_img

# Ensure we're in the correct directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

pregenual_path = 'masks/pregenual.nii.gz'
subgenual_path = 'masks/subgenual.nii.gz'

pregenual_img = load_image(pregenual_path)
subgenual_img = load_image(subgenual_path)

pregenual_data = pregenual_img.get_fdata()
subgenual_data = subgenual_img.get_fdata()

# We want to combine these two masks into a single mask. We'll call this mask 'anterior_cingulate'
anterior_cingulate_data = np.zeros_like(pregenual_data)
anterior_cingulate_data[pregenual_data == 1] = 1
anterior_cingulate_data[subgenual_data == 1] = 1

anterior_cingulate_img = load_image(anterior_cingulate_data)
anterior_cingulate_img.to_filename('masks/anterior_cingulate.nii.gz')

**Create amygdala mask**

In [11]:
import os
import numpy as np
import pandas as pd
from nilearn import image
from nimlab.jax_functions import load_mask, load_image, NiftiMasker
from numpy.linalg import inv
from nilearn.image import resample_to_img, load_img

# Ensure we're in the correct directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

path_1mm = 'masks/AAL3_1mm.nii.gz'
img = load_img(path_1mm)
# We only want to keep values in img where value is 149 or 150. We need to keep the same shape.
# Where its 149 or 150, set to 1, else 0
img_data = img.get_fdata()
new_data = np.zeros_like(img_data)
new_data[(img_data == 43) | (img_data == 44)] = 1
new_img = image.new_img_like(img, new_data)

mask_img = load_mask('MNI152_T1_2mm_brain_mask_dil')

resampled_img = resample_to_img(new_img, mask_img, interpolation='nearest')
resampled_img.to_filename('masks/amygdala.nii.gz')

**Create Caudate Mask**

In [16]:
import os
import numpy as np
import pandas as pd
from nilearn import image
from nimlab.jax_functions import load_mask, load_image, NiftiMasker
from numpy.linalg import inv
from nilearn.image import resample_to_img, load_img

# Ensure we're in the correct directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

path_1mm = 'masks/AAL3_1mm.nii.gz'
img = load_img(path_1mm)
# We only want to keep values in img where value is 149 or 150. We need to keep the same shape.
# Where its 149 or 150, set to 1, else 0
img_data = img.get_fdata()
new_data = np.zeros_like(img_data)
new_data[(img_data == 73) | (img_data == 74)] = 1
new_img = image.new_img_like(img, new_data)

mask_img = load_mask('MNI152_T1_2mm_brain_mask_dil')

resampled_img = resample_to_img(new_img, mask_img, interpolation='nearest')
resampled_img.to_filename('masks/caudate.nii.gz')

**Check our Septum Mask**

In [14]:
septum_path = 'masks/Ch-123_bilateral-prob25.nii.gz'
septum_img = load_img(septum_path)

# Resample it to the brain mask
mask_img = load_mask('MNI152_T1_2mm_brain_mask_dil')
resampled_septum_img = resample_to_img(septum_img, mask_img, interpolation='nearest')
resampled_septum_img.to_filename('masks/septum.nii.gz')
